In [1]:
# # Import necessary libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# Load the dataset
df=pd.read_csv('../data/boston_housing.csv')
df=df.dropna()


In [6]:
# Identify and handle outliers (for simplicity, remove values outside the 1st and 99th percentile)
for col in df.columns:
    if df[col].dtype!='object':
        q_low=df[col].quantile(0.01)
        q_high=df[col].quantile(0.99)
        df=df[(df[col]>=q_low) & (df[col]<=q_high)]


In [9]:

## Encode categorical variables
df=pd.get_dummies(df,columns=['chas'],drop_first=True)

In [10]:
# Normalize/standardize numerical features
scaler=StandardScaler()
numerical_feature=df.columns[df.dtypes!='object']
numerical_feature

Index(['crim', 'zn', 'indus', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'b', 'lstat', 'medv', 'chas_1'],
      dtype='object')

In [11]:
df[numerical_feature]=scaler.fit_transform(df[numerical_feature])

In [12]:
# Split the data into training and testing sets
x=df.drop('medv',axis=1)
y=df['medv']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [13]:
#Save preprocessed data
x_train.to_csv('../data/x_train.csv',index=False)
x_test.to_csv('../data/x_test.csv',index=False)
y_train.to_csv('../data/y_train.csv',index=False)
y_test.to_csv('../data/y_test.csv',index=False)